# Concatenando múltiplas bases



Em algumas situações, a informação que se precisa está dividida em bases de dados com formato idêntico. 

Nesses casos, podemos **concatenar** essas bases. 

O exemplo a seguir investiga a diversidade nos cursos da UFRN.

Nossa fonte de informação para este exemplo são os datasets sobre discentes ingressantes na UFRN, que estão organizados por ano de ingresso. 

## Coletando os dados

O primeiro passo quando temos bases de diferentes fontes é coletá-las da forma o mais automatizada possível.

Neste exemplo, organizamos os links para os dados de discentes em um dicionário.

No exemplo a seguir, cada chave é um ano e seu valor associado é o link para o dataset correspondente:

In [0]:
csv_discentes = {
    2019: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv",
    2018: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/146b749b-b9d0-49b2-b114-ac6cc82a4051/download/discentes-2018.csv",
    2017: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dc732572-a51a-4d4a-a39d-2db37cbe5382/download/discentes-2017.csv",
    2016: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/7d2fa5b3-743f-465f-8450-91719b34a002/download/discentes-2016.csv",
    2015: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/e2b5b843-4f58-497e-8979-44daf8df8f94/download/discentes-2015.csv",
    2014: "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/6c23a430-9a7c-4d0f-9602-1d5d97d40e6a/download/discentes-2014.csv",
}

Podemos consumir cada um desses datasets usando o método `pd.read_csv()`.

O Python permite que esse método seja aplicado a todos os links em sequência usando **compreensão de listas**, com a notação:

```python3
pd.read_csv(dicionário[chave]) for chave in dicionário
```

Entendendo o código acima, pedimos que o Python aplique o método `pd.read_csv(dicionário[chave])` para cada chave presente no dicionário.

A lista produzida contém um `DataFrame` para cada ano.

O método `pd.concat()` combina todos esses dataframes em um só:

In [0]:
import pandas as pd

In [0]:
dados = pd.concat(pd.read_csv(csv_discentes[ano], sep=";") for ano in csv_discentes)
dados

Agora que já unimos os datasets, vamos salvá-lo para não precisarmos refazer esse processo a cada vez.

Uma das formas de **persistir** um `DataFrame` é usar o método `to_csv()` que salva os dados em um arquivo CSV:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
dados.to_csv("/content/drive/My Drive/discentes-ufrn-2014-2019.csv", index=False)

## Avaliando a diversidade no IMD

Começamos nossa análise pelos dados do IMD.

Fazemos essa seleção filtrando pelo campo `nome_unidade`:

In [0]:
dados_imd = dados.query("nome_unidade == 'INSTITUTO METROPOLE DIGITAL'")
dados_imd.shape

Uma operação bastante comum em bancos de dados é calcular estatísticas que exigem **agrupar** os dados.

No Pandas, podemos fazer isso usando o método `groupby()` dos objetos do tipo `DataFrame`.

O exemplo a seguir calcula a quantidade de observações por ano, isto é, a quantidade de ingressantes nos cursos do IMD entre 2014 e 2019.

Antes, no entanto, precisamos verificar se há **dados faltando** nesta coluna:

In [0]:
dados_imd.isnull().sum()

Como não temos dados faltando, podemos prosseguir com a análise:

In [0]:
dados_imd.groupby("ano_ingresso").size()

Também podemos fazer agrupamentos considerando múltiplas características dos dados:

In [0]:
dados_imd.groupby(["nome_curso","ano_ingresso","sexo"]).size()

Para verificar existência de **dados inválidos**, precisaremos nos assegurar que fiquem no dataframe apenas observações que respeitem as possibilidades disponíveis para esta característica: 

In [0]:
dados_imd = dados_imd.query("sexo in ['M','F']")
dados_imd.shape

In [0]:
dados_imd.groupby(["nome_curso","ano_ingresso","sexo"]).size()

Vamos restringir nossa análise ao Bacharelado em Tecnologia da Informação (BTI).

Fazemos isso filtrando os dados pela característica `nome_curso`:

In [0]:
dados_bti = dados_imd.query("nome_curso == 'TECNOLOGIA DA INFORMAÇÃO'")

In [0]:
agregado_bti = dados_bti.groupby(["ano_ingresso","sexo"]).size()
agregado_bti

Como a quantidade de ingressantes por ano varia, comparar anos diferentes só é possível em termos relativos.

Para isso, o Pandas oferece o método `crosstab`, cuja opção `normalize` determina se a contagem de frequência será normalizada ou não e como. 

Neste caso, informamos que a normalização seja feita pela característica listada horizontalmente (índices):

In [0]:
percentual_bti = pd.crosstab(dados_bti["ano_ingresso"], dados_bti["sexo"], normalize="index")
percentual_bti

Note que agora a soma de cada valor possível para a característica `sexo` em um dado ano é igual a 1 (que representa 100% dos dados daquele ano).

Não estamos muito bem em diversidade no BTI, hein? :(
    
Vamos gerar um gráfico de linhas para visualizar a evolução dessa (falta de) diversidade ao longo dos anos considerados.

Para isso, além da `matplotlib`, vamos precisar da bibioteca `seaborn`.

Por convenção, importamos a biblioteca `seaborn` como `sns` e usamos o método `set()` para ativar suas configurações padrão.

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Produzimos o gráfico de linha usando o método `lineplot`, onde devemos informar através do argumento `data` os dados que pretendemos plotar.

Além disso, vamos associar um nome ao gráfico de linha para poder configurar o intervalo usado no eixo y através do método `set(ylim=(início, fim))`:

In [0]:
percentual_bti_feminino = percentual_bti["F"]
plot_bti = sns.lineplot(data=percentual_bti_feminino)
plot_bti.set(ylim=(0,1))
plt.xlabel("Ano")
plt.ylabel("Percentual")
plt.title("Discentes do sexo feminino por ano de ingresso")

## Avaliando os cursos associados ao BTI

O dado acima claramente é insatisfatório, mas precisamos analisá-lo em perspectiva.

Vamos compará-lo inicialmente com cursos de segundo ciclo que se seguem ao BTI:

In [0]:
cursos_ti = ["TECNOLOGIA DA INFORMAÇÃO","ENGENHARIA DE SOFTWARE","CIÊNCIA DA COMPUTAÇÃO"]
dados_ti = dados.query(f"nome_curso in {cursos_ti}")

Para começar, vamos conferir se os valores presentes neste dataframe respeitam as possibilidades elencadas para a característica `sexo`:

In [0]:
dados_ti.isnull().sum()

In [0]:
dados_ti.sexo.unique()

Agora podemos calcular os valores absolutos e relativos para cada um desses cursos:

In [0]:
agregado_ti = dados_ti.groupby(['nome_curso','ano_ingresso','sexo']).size()
agregado_ti

In [0]:
percentual_ti = pd.crosstab([dados_ti['nome_curso'], dados_ti['ano_ingresso']], dados_ti['sexo'], normalize="index")
percentual_ti

Note que para os cursos de ciência da computação e engenharia de software, não há dados para os anos de 2014 e 2015. 

Além disso, não houve ingresso de discentes do sexo feminino em ciência da computação no ano de 2019.

Vamos então considerar os dados para o sexo masculino a partir de 2016:

In [0]:
percentual_ti_masculino = percentual_ti.query("ano_ingresso >= 2016")["M"]
percentual_ti_masculino

Uma série produzida a partir de um agrupamento de múltiplas características apresenta um índice hierárquico.

O Pandas permite converter uma série assim em um dataframe usando o método `reset_index()`.

Os nomes das colunas são reutilizados do índice hierárquico. A última coluna pode ser nomeada usando o argumento `name`):

In [0]:
dados_ti_masculino = percentual_ti_masculino.reset_index(name="total")
dados_ti_masculino

Com os dados organizados, podemos agora calcular a média de participação masculina nos cursos associados ao BTI considerando o período 2016-2019:

In [0]:
dados_ti_masculino.groupby("nome_curso").mean()

Note que também está sendo calculada a média da coluna `ano_ingresso`.

Isso acontece porque, ao ler os datasets da UFRN, o Pandas entendeu que deveria configurar esta coluna como numérica.

Se quisermos impedir isso, devemos solicitar ao Pandas que trate essa coluna como uma característica ordinal:

In [0]:
dados_ti_masculino.ano_ingresso = dados_ti_masculino.ano_ingresso.astype(str)
dados_ti_masculino.groupby("nome_curso").mean()

Na média, os três cursos apresentam diversidade insatisfatória. No entanto, fica claro que os cursos de segundo ciclo apresentam uma situação menos crítica que o BTI.

Vamos olhar a evolução deste fator nos três cursos usando o `lineplot()` mais uma vez.

Neste exemplo, vamos configurar o tamanho do gráfico usando o método `plt.figure(figsize=(largura, altura))`.

In [0]:
plt.figure(figsize=(6, 6))
plot_it = sns.lineplot(x="ano_ingresso", y="total", hue="nome_curso", data=dados_ti_masculino)
plot_it.set(ylim=(0,1))
plt.xlabel("Ano")
plt.ylabel("Percentual")
plt.title("Discentes do sexo masculino por ano de ingresso")

## Avaliando a diversidade na UFRN

Como podemos ver, a diversidade nos cursos relacionados ao BTI não anda bem.

Mas será que essa é a realidade mais geral da UFRN?

Vamos expandir nossa análise para considerar todas as unidades acadêmicas da universidade.

Novamente, começamos verificando a existência de dados faltando:

In [0]:
dados.isnull().sum()

Quando identificamos a ausência de dados, podemos tentar preenchê-los ou descartar as observações afetadas.

Pela elevada quantidade de dados faltando, tentar preencher esses dados se torna difícil.

Assim, vamos optar por analisar apenas os casos em que o nome da unidade foi informada:

In [0]:
dados_não_nulos = dados[~dados["nome_unidade"].isnull()]

Outro recorte importante é limitar nossa análise a cursos de graduação:

In [0]:
dados_graduação = dados_não_nulos.query("nivel_ensino == 'GRADUAÇÃO'")

Após esses filtros, vamos ver quantas unidades restaram.

Podemos fazer isso usando o procedimento `len`, que conta a quantidade de elementos em uma lista:

In [0]:
nomes_unidades = dados_graduação["nome_unidade"].unique()
len(nomes_unidades)

In [0]:
nomes_unidades

Ainda no etapa de validação de dados, vamos ver se há dados inválidos na característica `sexo`.

Podemos fazer isso usando o procedimento `all(condição)`, que avalia se não houve quebra da condição informada: 

In [0]:
condição_sexo = dados_graduação["sexo"].isin(["M","F"])
all(condição_sexo)

Mantemos, então, apenas as entradas válidas:

In [0]:
dados_graduação = dados_graduação[condição_sexo]

Baseado nas experiências anteriores, qual o próximo passo em nossa análise?

In [0]:
percentual_graduação = pd.crosstab(dados_graduação["nome_unidade"], dados_graduação["sexo"], normalize="index")
percentual_graduação

Vamos isolar apenas os dados referentes ao público feminino.

Além disto, vamos ordenar estes valores em ordem crescente usando o método `sort_values()`:

In [0]:
dados_graduação_feminino = percentual_graduação["F"].sort_values()
dados_graduação_feminino

Os valores mínimo e máximo para o percentual de mulheres estão bastante separados.

Vamos dar uma olhada em estatísticas descritivas sobre essa série:

In [0]:
dados_graduação_feminino.describe()

A média e a mediana encontram-se próximas a 50%, o que é um dado interessante.

No entanto, o desvio padrão de 20% é bastante elevado.

Vamos ver quais cursos ficaram abaixo do primeiro quartil:

In [0]:
dados_graduação_feminino[dados_graduação_feminino <= dados_graduação_feminino.quantile(0.25)]

Em geral, são cursos relacionados a ciências exatas, engenharia e tecnologias. 

Essa é uma realidade conhecida, mas surpreende que cursos de música também estejam nesse grupo.

E quais ficaram acima do terceiro quartil?

In [0]:
dados_graduação_feminino[dados_graduação_feminino >= dados_graduação_feminino.quantile(0.75)]

Cursos da área de sáude e educação, outra realidade assimilada na universidade.

Para termos a visão completa, vamos usar o método `distplot` do seaborn para analisar a distribuição dos dados e compará-la com uma distribuição normal.

Para isso, precisaremos do método norm do da biblioteca `scipy`:

In [0]:
from scipy.stats import norm

In [0]:
sns.distplot(dados_graduação_feminino, fit=norm, bins=10)
plt.ylabel("Quantidade de unidades")
plt.xlabel("Percentual")
plt.title("Discentes de graduação do sexo feminino por unidade acadêmica")

Entendendo o código acima, a opção `fit=norm` gera a distribuição normal de referência para comparação, em preto.

Por sua vez, a opção `bins=10` configura quantos intervalos serão considerados pelo histograma.

Interpretando o gráfico, chegamos à hipotese de que haja três grandes grupos de cursos, sendo que apenas um representa uma situação de diversidade.

Vamos tentar avaliar essa hipótese com um maior refinamento, partindo para uma análise dos cursos da UFRN:

In [0]:
len(dados_graduação["nome_curso"].unique())

Aqui cabe uma ressalva: os datasets apontam 82 nomes de cursos diferentes, mas existem mais de 120 cursos de graduação na UFRN.

O motivo para o número menor é que cursos de licenciatura e bacharelado não são distinguidos pelo nome nestes datasets.

Continuando nossa análise:

In [0]:
percentual_cursos = pd.crosstab(dados_graduação["nome_curso"], dados_graduação["sexo"], normalize="index")
percentual_cursos

Selecionando apenas os dados para o público feminino e os ordenando:

In [0]:
dados_cursos_feminino = percentual_cursos["F"].sort_values()
dados_cursos_feminino

Novamente, cursos de engenharias, tecnologia, saúde e educação aparecem como os menos diversos.

Vamos analisar algumas estatísticas descritivas dessa série:

In [0]:
dados_cursos_feminino.describe()

Mais uma vez, a média e o desvio padrão estão próximas a 50% e o desvio padrão na casa de 20%.

Vamos ver quais os cursos abaixo do primeiro quartil:

In [0]:
dados_cursos_feminino[dados_cursos_feminino <= dados_cursos_feminino.quantile(0.25)]

De fato, o único novo insight que obtemos é a presença do curso de educação física neste grupo, uma exceção entre os cursos de saúde.

Vamos ver agora os cursos acima do terceiro quartil:

In [0]:
dados_cursos_feminino[dados_cursos_feminino >= dados_cursos_feminino.quantile(0.75)]

Aqui vemos mais uma exceção, com arquitetura e urbanismo neste grupo.

Além disso, chama a atenção a presença de cursos de Letra, considerando que sua unidade responsável apresenta proporção quase igualitária entre discentes homens e mulheres. 

Para finalizar, vamos usar o `distplot` e comparamos a distribuição dos dados com uma distribuição normal:

In [0]:
sns.distplot(dados_cursos_feminino, fit=norm, bins=10)
plt.ylabel("Quantidade de cursos")
plt.xlabel("Percentual")
plt.title("Discentes de graduação do sexo feminino por curso")

Neste caso, notamos uma distribuição bem mais aproximada da normal.

No entanto, há um pico considerável na extremidade inferior do eixo x.

No geral, podemos dizer que a diversidade na universidade está em uma situação melhor do que a diversidade nos cursos da universidade.

Em outras palavras, diferentes perfis encontrarão cursos para lhes atender, mas poucos cursos atenderão os diferentes perfis.